In [1]:
import alpaca_trade_api as tradeapi
#import alpaca_trade_api.polygon as polygon
import numpy as np
#import pandas as pd
#import yahoo_fin.stock_info as si
import time
import pandas as pd
import os
import requests
import bs4 as bs
import lxml
import requests
import subprocess
import pickle

In [4]:
# Initialize variables

#api = tradeapi.REST(key_id='PKHE6FHRIO5DS3TY47FS',secret_key='/r1wAfhxP2tg4cpoygUxlzoS0UvyJRmQ3nKtkI5J', base_url='https://paper-api.alpaca.markets')
api = tradeapi.REST('AKWQS0DX9ISRL37TSREO', 'wKaMSsjuHn44BzD15Wc5gqTRHobexAAuwPex7LLn')
os.environ["POLYGON_KEY_ID"] = "Z2pGestGKh7_x_G1r8_PAf_i4oVzp3z2Nm_QGB"
#print(os.environ)

In [5]:
account = api.get_account()
rest_response = api.polygon.financials_v2(symbol="MSFT", limit=2, report_type=tradeapi.polygon.rest.FinancialsReportType.Y , sort=tradeapi.polygon.rest.FinancialsSort.CalendarDateDesc)

In [69]:

##### CODE TO CALL CLASS ATTRIBUTES WITHOUT HAVING TO HARDCODE THEM #####
columns = ['cashAndEquivalents', 'debtToEquityRatio', 'earningsPerBasicShare','freeCashFlowPerShare', 'grossProfit', 'marketCapitalization', 'netCashFlow', 'netIncome', 'operatingExpenses', 'priceSales','researchAndDevelopmentExpense', 'revenues', 'salesPerShare', 'totalLiabilities', 'profitMargin']
for attr in columns:
    my_attr = getattr(rest_response[0], attr)
    print(attr,":",my_attr)

#########################################################################



cashAndEquivalents : 48844000000
debtToEquityRatio : 2.741
earningsPerBasicShare : 11.97
freeCashFlowPerShare : 12.754
grossProfit : 98392000000
marketCapitalization : 1105306601400
netCashFlow : 24311000000
netIncome : 55256000000
operatingExpenses : 34462000000
priceSales : 4.248
researchAndDevelopmentExpense : 16217000000
revenues : 260174000000
salesPerShare : 56.341
totalLiabilities : 248028000000
profitMargin : 0.212


In [6]:
def save_sp500_tickers():
   
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    tickers = map(lambda s: s.strip(), tickers)
    return list(tickers)

In [7]:
universe = save_sp500_tickers()
print(len(universe))

505


In [ ]:
columns = ['bookValuePerShare', 'cashAndEquivalents', 'debtToEquityRatio', 'earningsPerBasicShare','freeCashFlowPerShare', 'grossProfit', 'marketCapitalization', 'netCashFlow', 'netIncome', 'operatingExpenses', 'priceSales','researchAndDevelopmentExpense', 'revenues', 'salesPerShare', 'totalLiabilities', 'profitMargin']
#columns = ['grossProfit','cashAndEquivalents']
stock_df = pd.DataFrame([],columns=columns)
Yr1 = pd.DataFrame([],columns=columns)
Yr2 = pd.DataFrame([],columns=columns)

stocks = universe
for i, stock in enumerate(stocks):
    print(i, stock)
    rest_response = api.polygon.financials_v2(symbol=stock, limit=2, report_type=tradeapi.polygon.rest.FinancialsReportType.Y , sort=tradeapi.polygon.rest.FinancialsSort.CalendarDateDesc)
    y1 = []
    y2 = []
    for feature in columns:
        try:
            y1.append(getattr(rest_response[0], feature))
        except:
            y1.append(np.nan)

        try:
            y2.append(getattr(rest_response[1], feature))
        except:
            y2.append(np.nan)

    #print(y1)
    y1_df = pd.DataFrame([y1], columns = columns, index = [stock])
    y2_df = pd.DataFrame([y2], columns = columns, index = [stock])
    Yr1 = Yr1.append(y1_df)
    Yr2 = Yr2.append(y2_df)
    #display(y1_df-y2_df)
    stock_df = stock_df.append(y1_df-y2_df)
    time.sleep(20)
#display(stock_df)
stock_df.loc[stock_df.marketCapitalization>0,'label'] = 1
stock_df.loc[stock_df.marketCapitalization<=0,'label'] = 0
stock_df.drop(['marketCapitalization'],axis=1,inplace=True)
display(stock_df)
stock_df.to_csv('sample_stock_data.csv')
Yr1.to_csv('Yr1.csv')
Yr2.to_csv('Yr2.csv')

0 MMM
1 ABT
2 ABBV
3 ABMD
4 ACN
5 ATVI
6 ADBE
7 AMD
8 AAP
9 AES
10 AFL
11 A
12 APD
13 AKAM
14 ALK
15 ALB
16 ARE
17 ALXN
18 ALGN
19 ALLE
20 LNT
21 ALL
22 GOOGL
23 GOOG
24 MO
25 AMZN
26 AMCR
27 AEE
28 AAL
29 AEP
30 AXP
31 AIG
32 AMT
33 AWK
34 AMP
35 ABC
36 AME
37 AMGN
38 APH
39 ADI
40 ANSS
41 ANTM
42 AON
43 AOS
44 APA
45 AAPL
46 AMAT
47 APTV
48 ADM
49 ANET
50 AJG
51 AIZ
52 T
53 ATO
54 ADSK
55 ADP
56 AZO
57 AVB
58 AVY
59 BKR
60 BLL
61 BAC
62 BK
63 BAX
64 BDX
65 BRK.B
66 BBY
67 BIO
68 BIIB
69 BLK
70 BA
71 BKNG
72 BWA
73 BXP
74 BSX
75 BMY
76 AVGO
77 BR
78 BF.B
79 CHRW
80 COG
81 CDNS
82 CZR
83 CPB
84 COF
85 CAH
86 KMX
87 CCL
88 CARR
89 CTLT
90 CAT
91 CBOE
92 CBRE
93 CDW
94 CE
95 CNC
96 CNP
97 CERN
98 CF
99 SCHW
100 CHTR
101 CVX
102 CMG
103 CB
104 CHD
105 CI
106 CINF
107 CTAS
108 CSCO
109 C
110 CFG
111 CTXS
112 CLX
113 CME
114 CMS
115 KO
116 CTSH
117 CL
118 CMCSA
119 CMA
120 CAG
121 COP
122 ED
123 STZ
124 COO
125 CPRT
126 GLW
127 CTVA
128 COST
129 CCI
130 CSX
131 CMI
132 CVS
133 DHI
134 DHR
1

In [14]:
stock_df.to_csv('sample_stock_data.csv')

In [27]:
#a = pd.DataFrame([[1,2,3]],columns=['a','b','c'],index=['MSFT'])
#print(a)

#print(y1)
x = pd.DataFrame([y1], columns=columns, index=['MSFT'])
display(x)

,BookValuePerShare,cashAndEquivalents,debtToEquityRatio,earningsPerBasicShare,freeCashFlowPerShare,grossProfit,marketCapitalization,netCashFlow,netIncome,operatingExpenses,priceSalesresearchAndDevelopmentExpense,revenues,salesPerShare,totalLiabilities,profitMargin
MSFT,NaN,133819000000,1.8,5.11,4.986,82933000000,1054144621760,-590000000,39240000000,39974000000,NaN,125843000000,16.401,184226000000,0.312


In [104]:
z = tradeapi.polygon.rest.FinancialsSort.CalendarDateAsc
print(z)

NameError: name 'value' is not defined

In [36]:

https://api.polygon.io/v2/reference/financials/%5B'GE'%5D?limit=5&type=Q&sort=reportPeriod&apiKey=Z2pGestGKh7_x_G1r8_PAf_i4oVzp3z2Nm_QGB
https://api.polygon.io/v2/reference/financials/AAPL?limit=5&type=Q&sort=reportPeriod&apiKey=*


quote = si.get_quote_table("aapl")
#print(quote)
print(os.environ)

environ({'ALLUSERSPROFILE': 'C:\\ProgramData', 'APPDATA': 'C:\\Users\\anurruti\\AppData\\Roaming', 'APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL': 'true', 'ASL.LOG': 'Destination=file', 'CHROME_CRASHPAD_PIPE_NAME': '\\\\.\\pipe\\crashpad_17244_FQUJTDFPIFXBQAHZ', 'COMMONPROGRAMFILES': 'C:\\Program Files\\Common Files', 'COMMONPROGRAMFILES(X86)': 'C:\\Program Files (x86)\\Common Files', 'COMMONPROGRAMW6432': 'C:\\Program Files\\Common Files', 'COMPUTERNAME': 'DESKTOP-RJ8GNQA', 'COMSPEC': 'C:\\WINDOWS\\system32\\cmd.exe', 'DRIVERDATA': 'C:\\Windows\\System32\\Drivers\\DriverData', 'ELECTRON_RUN_AS_NODE': '1', 'HOMEDRIVE': 'C:', 'HOMEPATH': '\\Users\\anurruti', 'JUPYTER_CONFIG_DATA': 'c:\\Users\\anurruti\\.vscode\\extensions\\ms-toolsai.vscode-ai-remote-0.1.0\\jupyterConfigData.json', 'LOCALAPPDATA': 'C:\\Users\\anurruti\\AppData\\Local', 'LOGONSERVER': '\\\\DESKTOP-RJ8GNQA', 'NUMBER_OF_PROCESSORS': '8', 'ONEDRIVE': 'C:\\Users\\anurruti\\OneDrive - Microsoft', 'ONEDRIVECOMMERCIAL': 'C:\\User

In [29]:
quote = si.get_stats_valuation("msft")
print(quote)
print(quote.iloc[:,1])

                    Unnamed: 0 As of Date: 4/6/2021Current 12/31/2020  \
0      Market Cap (intraday) 5                       1.88T      1.68T   
1           Enterprise Value 3                       1.82T      1.61T   
2                 Trailing P/E                       37.12      35.87   
3                Forward P/E 1                       30.86      33.00   
4  PEG Ratio (5 yr expected) 1                        2.29       2.57   
5            Price/Sales (ttm)                       12.42      11.59   
6             Price/Book (mrq)                       14.42      13.60   
7   Enterprise Value/Revenue 3                       11.85      37.42   
8    Enterprise Value/EBITDA 6                       24.14      74.38   

  9/30/2020 6/30/2020 3/31/2020  
0     1.59T     1.54T     1.20T  
1     1.53T     1.48T     1.14T  
2     36.52     33.92     27.48  
3     32.15     32.36     25.32  
4      2.50      2.48      2.77  
5     11.30     11.30      9.07  
6     13.45     13.46     10.87

### Scrap Work

In [11]:
rest_response = api.polygon.financials_v2(symbol="MSFT", limit=2, report_type=tradeapi.polygon.rest.FinancialsReportType.Y , sort=tradeapi.polygon.rest.FinancialsSort.CalendarDateDesc)

In [12]:
rest_response

[Financial({   'EBITDAMargin': 0.44,
     'accumulatedOtherComprehensiveIncome': -340000000,
     'accumulatedRetainedEarningsDeficit': 24150000000,
     'assetTurnover': 0.472,
     'assets': 286556000000,
     'assetsAverage': 266578750000,
     'assetsCurrent': 175552000000,
     'assetsNonCurrent': 111004000000,
     'averageEquity': 93822250000,
     'bookValuePerShare': 13.336,
     'calendarDate': '2019-12-31',
     'capitalExpenditure': -13925000000,
     'cashAndEquivalents': 133819000000,
     'cashAndEquivalentsUSD': 133819000000,
     'consolidatedIncome': 39240000000,
     'costOfRevenue': 42910000000,
     'currentLiabilities': 69420000000,
     'currentRatio': 2.529,
     'dateKey': '2019-08-01',
     'debt': 78366000000,
     'debtCurrent': 5516000000,
     'debtNonCurrent': 72850000000,
     'debtToEquityRatio': 1.8,
     'debtUSD': 78366000000,
     'deferredRevenue': 37206000000,
     'deposits': 0,
     'depreciationAmortizationAndAccretion': 11682000000,
     'divi

In [13]:
1:55e9, 58e9

SyntaxError: illegal target for annotation (<ipython-input-13-eefeca5db82a>, line 1)